# Flowers-102 Image Classification

## Imports

In [398]:
import torch
import torchvision
#following import name conventions
import torchvision.transforms.v2 as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Processor choice

In [399]:
#Use GPU if possible
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

## Loading and Normalising Data

In [400]:
#Transform to be ran on data
train_transforms = transforms.Compose([
    transforms.Resize((500,500)),
    transforms.RandomHorizontalFlip(), #reduce bias with image flip
    transforms.RandomRotation(10), #reduce bias with random angles
    
    transforms.ToTensor(),
    transforms.Normalize((0.5151, 0.4092, 0.3384), (0.2505, 0.2084, 0.2139)) #mean and std of dataset
])

test_transforms = transforms.Compose([
    transforms.Resize((500,500)),

    transforms.ToTensor(),
    transforms.Normalize((0.5151, 0.4092, 0.3384), (0.2505, 0.2084, 0.2139)) #mean and std of dataset
])

c:\Users\nicho\anaconda3\lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [401]:
#Hyperparameters
batch_size = 32
learning_rate = 0.01
epoch_count = 100
escape_loss = 0.01
val_batch_size = 28

In [402]:
train_data = torchvision.datasets.Flowers102(
    root="./data",
    download=True,
    split= "train",
    transform=train_transforms
)
validation_data = torchvision.datasets.Flowers102(
    root="./data",
    download=True,
    split= "val",
    transform=test_transforms
)
test_data = torchvision.datasets.Flowers102(
    root="./data",
    download=True,
    split= "test",
    transform=test_transforms
)

In [403]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size)
validation_loader = torch.utils.data.DataLoader(validation_data,val_batch_size,drop_last=True,)

## Convolutional Neural Network

In [404]:
class cnn(nn.Module):
    def __init__(self):
        #Layers
        
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # params are input channels, output channels, filter(kernel) size
        self.pool = nn.MaxPool2d(2,2) # params are kernal size, stride
        self.conv2 = nn.Conv2d(6, 5, 5)

        self.lin1 = nn.Linear(74420,400) # params are input features, output features
        self.lin2 = nn.Linear(400, 200)
        self.lin3 = nn.Linear(200,102) #output features must be equal to num of categories
    
    def forward(self,x):
        """Called to pass input data through layers, uses activation function"""
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, 1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x

model = cnn()



## Loss function and optimizer

In [405]:
loss_fn = nn.CrossEntropyLoss() #  probibalistic good for multiclass uses softmax
optimizer = optim.SGD(model.parameters(),learning_rate) #Stochastic gradient descent

## Training loop

In [406]:
escape_loop = False
for epoch in range(epoch_count): #loop multiple times
    model.train()
    running_loss = 0.0
    val_running_loss = 0.0
    if escape_loop == True:
        escape_loop = False
        break
    for batch, (images,labels) in enumerate(train_loader): # pulls out inputs and labels from training data
        optimizer.zero_grad() #zero parameter gradients
        
        #forward cnn, backpropagation (backward and optimize)
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        
        #print statistics
        running_loss += loss.item()
        if((batch+1)%8 == 0):
            print(f"[Epoch: {epoch + 1}, Batch progress: {batch + 1:5d}] loss: {running_loss/8:.3f}") 
            running_loss = 0.0

    #Each epoch calculate validation loss as to know if to break out due to overfitting
    model.eval()
    with torch.no_grad(): # in validation, don't need to calculate gradients for ouputs
        for i,(val_images,val_labels) in enumerate(validation_loader):
            val_outputs = model(val_images)
            val_loss = loss_fn(val_outputs,val_labels)
            val_running_loss += val_loss.item()
        print(f"Validation loss: {val_running_loss/count:.3f}")
    
    #Breakout clause
    
print("Training complete")

[Epoch: 1, Batch progress:     8] loss: 4.623
[Epoch: 1, Batch progress:    16] loss: 4.627
[Epoch: 1, Batch progress:    24] loss: 4.624
[Epoch: 1, Batch progress:    32] loss: 4.626
Validation loss: 4.622
[Epoch: 2, Batch progress:     8] loss: 4.619
[Epoch: 2, Batch progress:    16] loss: 4.621
[Epoch: 2, Batch progress:    24] loss: 4.622
[Epoch: 2, Batch progress:    32] loss: 4.615
Validation loss: 4.614
[Epoch: 3, Batch progress:     8] loss: 4.608
[Epoch: 3, Batch progress:    16] loss: 4.605
[Epoch: 3, Batch progress:    24] loss: 4.603
[Epoch: 3, Batch progress:    32] loss: 4.593
Validation loss: 4.581
[Epoch: 4, Batch progress:     8] loss: 4.552
[Epoch: 4, Batch progress:    16] loss: 4.555
[Epoch: 4, Batch progress:    24] loss: 4.529
[Epoch: 4, Batch progress:    32] loss: 4.502
Validation loss: 4.452
[Epoch: 5, Batch progress:     8] loss: 4.396
[Epoch: 5, Batch progress:    16] loss: 4.411
[Epoch: 5, Batch progress:    24] loss: 4.314
[Epoch: 5, Batch progress:    32] 

## Test model

In [ ]:
correct = 0
total = 0
accuracy = 0
model.eval()
with torch.no_grad(): # in testing, don't need to calculate gradients for ouputs
    for (images,labels) in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1) # returns predicted class labels in one dimension
        total += labels.size(0) #adds number of samples in batch to total sample count
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Accuracy of the network on the test images: {test_accuracy:.5f} %")

Accuracy of the network on the test images: 17.25484 %


## Check for overfitting

In [ ]:
correct = 0
total = 0
accuracy = 0

with torch.no_grad(): # in testing, don't need to calculate gradients for ouputs
    for (images,labels) in train_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1) # returns predicted class labels in one dimension
        total += labels.size(0) #adds number of samples in batch to total sample count
        correct += (predicted == labels).sum().item()

train_accuracy = 100 * correct / total
print(f"Accuracy of the network on the training images: {train_accuracy:.5f} %")

Accuracy of the network on the training images: 99.70588 %


## Save model

In [ ]:
torch.save(model.state_dict(), f"models/model_accuracy_{test_accuracy:.5f}.pt")